In [1]:
import os
import glob

import pandas as pd
import numpy as np

from nltk.tokenize import word_tokenize

from sklearn.ensemble import AdaBoostClassifier
from sklearn import svm
from sklearn.model_selection import train_test_split, KFold
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_curve, auc, confusion_matrix

%matplotlib inline

In [2]:
data_path = 'data'
files = glob.glob(os.path.join(data_path, '*', '*'))

In [3]:
train = pd.read_csv(files[3])

In [4]:
train.reset_index(drop = True, inplace = True)

In [5]:
stopwords = open('stopwords-id.txt', 'r').read().splitlines()

def cleansing(sentence):
    sentence = sentence.lower()
    word_list = word_tokenize(sentence)
#     word_list = [word for word in word_list if len(word) > 2 and word not in stopwords]
    word_list = [word for word in word_list if len(word) > 2]
    sentence = " ".join(word_list)
    
    return sentence

train['response_cleansing'] = train.RESPONSE.apply(cleansing)

In [7]:
# train
# train.groupby('LABEL').count()

In [11]:
# Train, Test = train_test_split(train, test_size=0.2, stratify=train['LABEL'], random_state=0)

# cv = CountVectorizer(analyzer="word", ngram_range=(1,1), stop_words=stopwords, lowercase=True)

# train_vect = cv.fit_transform(Train['response_cleansing'].astype(str)) #fit transform pada data train A
# test_vect = cv.transform(Test['response_cleansing'].astype(str)) #transform data test

# mnb = MultinomialNB()
# mnb.fit(train_vect, Train['LABEL'])
# res_cv = mnb.predict(test_vect)

# print("accuracy", accuracy_score(res_cv, Test['LABEL']))
# print("f1 score", f1_score(res_cv, Test['LABEL']))
# print("precision", precision_score(res_cv, Test['LABEL']))
# print("recall", recall_score(res_cv, Test['LABEL']))
# tn_dev, fp_dev, fn_dev, tp_dev = confusion_matrix(res_cv, Test['LABEL']).ravel()
# tpr = tp_dev/(tp_dev+fn_dev)
# fpr = fp_dev/(fp_dev+tn_dev)
# print("tpr", tpr)
# print("fpr", fpr)

accuracy 0.639344262295082
f1 score 0.725
precision 0.8529411764705882
recall 0.6304347826086957
tpr 0.6304347826086957
fpr 0.3333333333333333


/home/arynas/.virtualenvs/ugm-ukara/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['baiknya', 'berkali', 'kali', 'kurangnya', 'mata', 'olah', 'sekurang', 'setidak', 'tama', 'tidaknya'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [ ]:
# Feature Extraction with Count Vectorizer
cv = CountVectorizer(analyzer="word", ngram_range=(1,1), stop_words=stopwords, lowercase=True)
# X = cv.fit_transform(train.response_cleansing)
X = cv.fit_transform(train.response_cleansing.astype(str))
# Y = train.LABEL
Y = cv.transform(train.LABEL.astype(str))

# # Feature Extraction with TF IDF
# tfidf = TfidfVectorizer()
# # X = tfidf.fit_transform(train.response_cleansing)
# X = tfidf.fit_transform(train.response_cleansing.astype(str))
# # Y = train.LABEL
# Y = tfidf.transform(train.LABEL.astype(str))

def evaluation(pred, y_label):
    accuracy = accuracy_score(pred, y_label)
    f1score = f1_score(pred, y_label)
    prec = precision_score(pred, y_label)
    recall = recall_score(pred, y_label)
#     fpr, tpr, thresholds = roc_curve(pred, y_label)

    tn_dev, fp_dev, fn_dev, tp_dev = confusion_matrix(pred, y_label).ravel()
    tpr = tp_dev/(tp_dev+fn_dev)
    fpr = fp_dev/(fp_dev+tn_dev)
    
#     return {'f1score': f1score, 'precision': prec, 'recall': recall, 'fpr': fpr[1], 'tpr': tpr[1]}
    return {'accuracy': accuracy, 'f1score': f1score, 'precision': prec, 'recall': recall, 'fpr': fpr, 'tpr': tpr}

kf = KFold(n_splits=5)

score = []
for tr, ts in kf.split(X, Y):
    X_train, y_train = X[tr], train.loc[tr, 'LABEL']
    X_test, y_test = X[ts], train.loc[ts, 'LABEL']
    
#     mnb = MultinomialNB()
#     mnb.fit(X_train, y_train)
#     res = mnb.predict(X_test)
    
#     svc = svm.SVC(gamma=0.01, C=100., probability=True)
#     svc.fit(X_train, y_train)
#     res = svc.predict(X_test)

    adb = AdaBoostClassifier()
    adb.fit(X_train , y_train)
    res = adb.predict(X_test)
    
    score.append(evaluation(res, y_test))
    
pd.DataFrame(score)